<div>
  <div>
    Emme Notebook and Scripting Course, August 2019 <br>
  </div>
  <div>
    <img style="align: left; margin: 15px 15px 15px 0px;" src="./INRO Logo.png" width="120" />
  </div>
  <div>
    Â© Copyright 2019 INRO
  </div>
</div>

# 2. Network API - Read, Write and Process Network Data

This class will guide you through how to read, manipulate and publish Emme network data using the Emme Network API. We will learn to write scripts using the Network API to create, modify or delete modes, transit vehicles, nodes, links, turns, transit lines and transit segments. Accessing network data through the Network API also allows you to implement more complex calculations that could not be done in one step in the Network Calculator. For a full description of the Emme Network API modules, methods and classes, refer to the Emme API Reference.

__Suggested Duration__: 2 hours

## 2.1 Contents

<a href="#2.1-Contents">2.1 Contents</a>

<a href="#2.2-Iterating-through-network-elements">2.2 Iterating through network elements</a>

<a href="#2.3-Accessing-network-elements">2.3 Accessing network elements</a>

<a href="#2.4-Creating-network-elements">2.4 Creating network elements</a>

<a href="#2.5-Deleting-network-elements">2.5 Deleting network elements</a>

<a href="#2.6-Accessing-and-modifying-standard-attributes,-result-attributes,-extra-attributes-and-network-fields">2.6 Accessing and modifying standard attributes, result-attributes, extra-attributes and network fields</a>

<a href="#2.7-Publishing-network-modifications">2.7 Publishing network modifications</a>

<a href="#2.8-Creating-new-extra-attributes-and-network-fields">2.8 Creating new extra-attributes and network fields</a>

<a href="#2.9-PRACTICE:-Network-API">2.9 PRACTICE: Network API</a>

<a href="#2.10-Additional-Example---Express-transit-lines">2.10 Additional Example - Express transit lines</a>

<a href="#2.11-Quick-copy-of-attribute-values-from-scenario/network-to-scenario/network">2.11 Quick copy of attribute values from scenario/network to scenario/network</a>

<a href="#2.12-Shortest-path-calculation">2.12 Shortest path calculation</a>

<a href="#2.13-Additional-Example---Display-shortest-path-calculation-results">2.13 Additional Example - Display shortest path calculation results</a>

<a href="#2.14-Speeding-up-network-data-processing-with-get_partial_network">2.14 Speeding up network data processing with get_partial_network</a>




Emme Network APIs are available on the <code><b>Network</b></code> object. The __`Network`__ object is an instance of the class <code><b>inro.emme.network.Network</b></code> which can be constructed from an existing scenario using the __`scenario.get_network()`__ method; where __`scenario`__ is an instance of class <code><b>inro.emme.database.scenario.Scenario</b></code>. In this example we will construct a network object from scenario 3001. 

In [ ]:
import inro.modeller as _m
modeller = _m.Modeller()

emmebank = modeller.emmebank
scenario_3001 = emmebank.scenario(3001)

In [ ]:
# the network is copied from the scenario
network_3001 = scenario_3001.get_network()

## 2.2 Iterating through network elements

From the <code><b>Network</b></code> object it is possible to access network elements using iterators. For example, the <code>network<b>.links()</b></code> iterator returns all the <code><b>inro.emme.network.link.Link</b></code> objects from the network. All the iterators available are:

- `network.modes()`
- `network.nodes()` - iterates over all regular nodes and centroids
- `network.regular_nodes()` - iterates over all non-centroids
- `network.centroids()` - iterates over all centroids
- `network.links()`
- `network.turns()`
- `network.transit_lines()`
- `network.transit_segments()`
- `network.transit_vehicles()`


In [ ]:
# Print out some attributes of the network nodes:
for node in network_3001.nodes():
    node_id = node.number
    if node_id < 5:
        print 'node', node.number
        print 'is_centroid', node.is_centroid
        print 'is_intersection', node.is_intersection
        print 'coordinates', node.x, node.y
        print 'initial_boardings', node.initial_boardings
        print 'final_alightings', node.final_alightings

In [ ]:
# Print out some attributes of link and link related nodes
count = 0
for link in network_3001.links():
    print link, link.modes, link.i_node, link.i_node.x, link.i_node.y, link.j_node
    count += 1
    if count >= 5:
        break

In [ ]:
# list and print out some transit vehicle attributes
for transit_vehicle in network_3001.transit_vehicles():
    print transit_vehicle, transit_vehicle.description

## 2.3 Accessing network elements
Specific network elements can be accessed through their  identifiers:

- `network.mode(id)`
- `network.node(id)`
- `network.link(i_node_id, j_node_id)`
- `network.turn(i_node_id, j_node_id, k_node_id)`
- `network.transit_line(id)`
- `network.transit_vehicle(id)`

In [ ]:
# Access node 1054 and print out its information
node_1054 = network_3001.node(1054)
print node_1054, node_1054.is_centroid, node_1054.is_intersection

In [ ]:
# Access transit line 15ae and print its segments with their transit volumes
transit_line_15ae = network_3001.transit_line('15ae')
for segment in transit_line_15ae.segments(include_hidden=True):
    print segment, segment.transit_volume

In [ ]:
# Identifying the link with the highest auto volume
max_volume_link = None

for link in network_3001.links():
    if max_volume_link is None:
        max_volume_link = link
    elif max_volume_link.auto_volume < link.auto_volume:
        max_volume_link = link
max_volume_link

## 2.4 Creating network elements
Network elements can be created using one of the following functions

- `network.create_mode(type, id)`
- `network.create_node(id, is_centroid)`
- `network.create_link(i_node_id, j_node_id)`
- `network.create_intersection(id)`
- `network.create_transit_vehicle(id, mode_id)`
- `network.create_transit_line(id, transit_vehicle_id, itinerary)`

Note that in order to create a transit line, you need to pass the `itinerary` of the line. An itinerary is an iterable of two or more regular node IDs. There must be a link between each pair of adjacent nodes in the itinerary. Ex: `itinerary=[603, 602, 601, 600]`


In [ ]:
# create a new mode
mode_boat = network_3001.create_mode('TRANSIT', 'B')
mode_boat.speed = 15
mode_boat.description = 'boat'

In [ ]:
# create a new transit vehicle
ferry = network_3001.create_transit_vehicle(30, 'B')
ferry.description = 'Ferry'
ferry.seated_capacity = 50
ferry.total_capacity = 500

In [ ]:
# Create a new node
node_9999 = network_3001.create_node(9999, is_centroid=False)
node_9999.x = 633948
node_9999.y = 5529406

In [ ]:
# Create a new link
link_9999_1 = network_3001.create_link(
    i_node_id=9999,
    j_node_id=1,
    modes=[mode_boat]
)

## 2.5 Deleting network elements
Network elements can be deleted using one of the following functions:

- `network.delete_mode(id, cascade)`
- `network.delete_node(id, cascade)`
- `network.delete_link(i_node_id, j_node_id, cascade)`
- `network.delete_intersection(id)`
- `network.delete_transit_vehicle(id, cascade)`
- `network.delete_transit_line(id)`

The argument `cascade` is boolean (`True` or `False`). If `cascade` is `False`, the network element is deleted only if it is not used by any other network elements. For instance, when trying to delete node 1054 with `cascade=False`, the operation will fail as node 1054 is used by links (783-1054), (1053-1054), (1054-783) and (1054-1053).

In [ ]:
network_3001.delete_node(1054)

If `cascade` is True, the network elements will be removed as well as all network elements which rely on it. For instance, removing node 600 will cause to remove 8 links, 20 transit lines and all their segments and 9 turning movements. The argument `cascade=True` should be used carefully.

## 2.6 Accessing and modifying standard attributes, result attributes, extra-attributes and network fields
Standard attributes and results attributes are Python parameters and can be accessed directly from the Python object as we have seen in previous examples using the `dot` notation. Note that results attributes are available only when the scenario has results.

Note that some attributes are renamed in the API as compared to Emme keyword. For instance link <code>volau</code> is referenced as <code>link<b>.auto_volume</b></code>. See the Emme API Reference for full details.


If traffic and transit assignments have been run on the current scenario, traffic and transit results are available. We can verify that this is the case with the following code:

In [ ]:
print scenario_3001.has_traffic_results
print scenario_3001.has_transit_results

Display the node number, `us1` and the result attribute `inboa` for node 9999.

In [ ]:
print node_9999.number
print 'ui1: ', node_9999.data1
print 'inboa: ', node_9999.initial_boardings

Extra attributes and network field values can be accessed using the square bracket notation.

In [ ]:
print '@nflag: ', node_1054['@nflag']
print '#landmark: ', node_1054['#landmark']

Standard attribute, extra-attribute and network field values can be modified as well using the square brackets notation and by assign them a value. Note that for standard attributes you can use the dot notation as well.

In [ ]:
node_1054.data2 = 10
node_1054['data1'] = 123
node_1054['@nflag'] = 2
node_1054['#landmark'] = 'Bridge'

Existing extra-attributes and network fields on a scenario can be listed using `scenario.extra_attributes()` and `scenario.network_fields()`.

In [ ]:
for extra_attribute in scenario_3001.extra_attributes():
    print extra_attribute

In [ ]:
for network_field in scenario_3001.network_fields():
    print network_field

The network field object *#transit_line* can be access through the `scenario.network_field(type, id)` method:

In [ ]:
station_network_field = scenario_3001.network_field('NODE', '#station')

In [ ]:
station_network_field

In [ ]:
station_network_field.description = 'Transit station name'

## 2.7 Publishing network modifications
Until now, we have modified the network stored in memory. If we want to publish these changes to disk, we must call the method  `scenario.publish_network(network)`. 

Before being written to disk, the network is validated and any unauthorized network state will generate a meaningful error. So far everything is ok to publsh.

In [ ]:
scenario_3001.publish_network(network_3001)

## 2.8 Creating new extra-attributes and network fields

It is important to understand the difference between attribute definitions at the __scenario__ and __network__ level.

- When you first load your network using `scenario.get_network()` the network attributes defined will match those of the source scenario.




- If you need to create new attribute on your network, you can do this using:
  - `network.create_attribute(type, id, default_value)`
  - `type` must be one of the following: `'MODE', 'TRANSIT_VEHICLE', 'NODE', 'LINK', 'TURN', 'TRANSIT_LINE'`, and `'TRANSIT_SEGMENT'`  

- But in order to publish attribute values to disk, the extra-attributes and network fields must be defined in the target scenario. This can be done with the Database API: 
  - `scenario.create_extra_attribute(type, id, default_value)`
  - `scenario.create_network_field(type, id, atype, description)` 
  - note that it can also be done using the corresponding Modeller tools

Tip: 
- If you know which attributes you will need beforehand, create them on the __scenario__ before you call `scenario.get_network()`, for instance:

In [ ]:
vcr_att = scenario_3001.create_extra_attribute('LINK', '@vcr2', 0)
road_name_att = scenario_3001.create_network_field('LINK', '#road_name', 'STRING', 'Road names')
network_3001 = scenario_3001.get_network()

If you try to publish a network with attributes that do not match the target scenario, the publish operation will fail. For example:


In [ ]:
network_3001.create_attribute('TRANSIT_SEGMENT', '@transit_load', 0)
network_3001.create_attribute('TRANSIT_LINE', '#line_name')
scenario_3001.publish_network(network_3001)

If you do not require to publish these network attributes, you can drop them. The `scenario.publish_network(network, resolve_attributes)` method offer an optional argument `resolve_attributes`. If `resolve_attributes` is `False`, an error will be raised if the network contains attributes which are not defined in the scenario, or if the network does not contain attributes that are defined in the scenario. If it is `True`, missing attributes will be dropped.

In [ ]:
scenario_3001.publish_network(network_3001, resolve_attributes=True)

If the new network attributes values need to be kept, you have to create the corresponding scenario attributes. For example:

In [ ]:
transit_load_att = scenario_3001.create_extra_attribute('TRANSIT_SEGMENT', '@transit_load', 0)
scenario_3001.create_network_field('TRANSIT_LINE', '#line_name', 'STRING', 'Transit line ID')
scenario_3001.publish_network(network_3001, resolve_attributes=False)

Extra-attribute and network field parameters can be changed like this:

In [ ]:
vcr_att.description = 'Volume over capacity ratio'

## 2.9 <span style="color:red">PRACTICE: Network API</span>

Please refer to the _Emme Notebook and Scripting - Practices_ Notebook to complete this exercise. Note that the solutions to practices are found in the _Emme Notebook and Scripting - Solutions_ Notebook.

## 2.10 Additional Example - Express transit lines
Say we want to modify properties of the Express mode (mode 'E') to reflect larger inter-station distances, faster speeds, higher seated vehicle capacity and an express stop pattern. 

On a fresh copy of scenario 3001 use the Network API to:

1. Modify the transit vehicle 11 - Express bus and set its seated capacity to 60.
2. Compute the average distance between transit stops for each transit lines and verify that it is a bigger value for Express transit lines.
3. Modify the express transit lines to only allow boardings on the first half of their itinerary and only allow alighting on the second half (only where stops are already existing)

### Modify transit vehicle 11

In [ ]:
network = scenario_3001.get_network()
express_bus = network.transit_vehicle(11)
express_bus.seated_capacity = 60

### Compute the average distance between transit stops for each transit lines

In [ ]:
# First example for 1 transit line
example_line = network.transit_line('15ae')

n_stops = 0
line_length = 0

for segment in example_line.segments(include_hidden=False):
    line_length += segment.link.length
    if segment.allow_alightings or segment.allow_boardings:
        n_stops += 1
        
line_length / (n_stops + 1) * 1000 # Convert from km to m

In [ ]:
# Iterate over all transit lines and report
for transit_line in network.transit_lines():
    n_stops = 0
    line_length = 0

    for segment in transit_line.segments(include_hidden=False):
        line_length += segment.link.length
        if segment.allow_alightings or segment.allow_boardings:
            n_stops += 1

    print transit_line.id, transit_line.mode, line_length / (n_stops + 1) * 1000

It turns out that the inter-station distance is not really different between Express and bus mode. We could now use the Network API to modify this.

### Modify boarding and alighting authorizations on Express transit lines

In [ ]:
# iterate through transit lines
for transit_line in network.transit_lines():
    # process only express transit lines
    if transit_line.mode.id == 'E':
        # compute the transit line length
        transit_line_length = 0
        for segment in transit_line.segments():
            transit_line_length += segment.link.length
        
        # Set the boarding and alighting authorizations for each segment depending on their position along the itinerary.
        distance_along_itinerary = 0
        for segment in transit_line.segments():
            if segment.allow_alightings or segment.allow_boardings:
                if distance_along_itinerary < transit_line_length / 2:
                    segment.allow_alightings = False
                    segment.allow_boardings = True
                else:
                    segment.allow_alightings = True
                    segment.allow_boardings = False         
            distance_along_itinerary += segment.link.length

### Publish the modifications

In [ ]:
scenario_3001.publish_network(network)

## 2.11 Quick copy of attribute values from scenario/network to scenario/network
There is a really fast way to copy attribute values from one scenario / network to another scenario / network:
- __`get_attribute_values()`__ 
- __`set_attribute_values()`__  

Note that this method is fast because it only reads the minimal amount of information, as opposed to load in the whole network.

These methods can be used between two scenarios, two networks, and between a scenario and a network. In this example, we will copy the `hdw` values and the `lay1` values from scenario 2000 to scenario 3001. 

If a get/set_attribute_values is used between two scenarios or networks that donâ€™t have the same topology for the given domain type, then only the values for those objects with identifiers that are common between both will be copied.

First verify the headway values for transit lines `27be`, `45be` and `56be` in scenario 3001:

In [ ]:
transit_lines = ['27be','45be','56be']
for line_id in transit_lines:
    line = network.transit_line(line_id)
    print "Line %s has a headway value of %s in scenario 3001" % (line.id, line.headway)

Access the network for scenario 2000 and verify the headway values for these transit lines:

In [ ]:
scenario2000 = emmebank.scenario(2000)
network2000 = scenario2000.get_network()

transit_lines = ['27be','45be','56be']
for line_id in transit_lines:
    line = network2000.transit_line(line_id)
    print "Line %s has a headway value of %s in scenario 2000" % (line.id, line.headway)

Copy the headway values from scenario 2000 with the __`get_attribute_values()`__ method:

In [ ]:
elem_type = "TRANSIT_LINE"
attributes = ["headway"]
values = network2000.get_attribute_values(elem_type, attributes)

Use the __`set_attribute_values()`__ method to copy the `hdw` values to scenario 3001:

In [ ]:
scenario_3001.set_attribute_values(elem_type, attributes, values)

## 2.12 Shortest path calculation
The Network API offers a shortest path tree calculation. It can compute the shortest path from ANY node to all other nodes. Build a shortest path tree and return a new shortest path tree object representing node-to-node shortest paths on the network. The shortest path method accepts the following arguments: `shortest_path_tree(origin_node_id, link_costs, exluded_links, consider_turns, turn_costs, max_cost)`

In [ ]:
network_3001 = emmebank.scenario(3001).get_network()

In [ ]:
shortest_path_tree = network_3001.shortest_path_tree(
    origin_node_id=1,
    link_costs='length',
    excluded_links=[network_3001.link(602, 601), network_3001.link(601, 602)],
    consider_turns=False
)
    

A ShortestPathTree provides methods for iterating over and querying costs and paths to all reachable nodes and links.
Access the list of reachable nodes and links:

In [ ]:
print list(shortest_path_tree.reachable_nodes())
print list(shortest_path_tree.reachable_links())

Get the cost to reach one node as well as its link.

In [ ]:
print shortest_path_tree.cost_to_node(146)
print shortest_path_tree.path_to_node(146)

List the nodes and cost to reach them.

In [ ]:
count = 0
for link_cost in shortest_path_tree.link_costs():
    print link_cost
    count += 1
    if count >= 5:
        break

## 2.13 Additional Example - Display shortest path calculation results
To do that you will need to:

- create an extra-attribute `@cost_from_1`
- perform a shortest path calculation
- set the `@cost_from_1` values
- display the values in the general worksheet

In [ ]:
# Create a new extra attribute
scenario_3001 = emmebank.scenario(3001)
scenario_3001.create_extra_attribute('NODE', '@cost_from_1', 0)

In [ ]:
# Compute the shortest path free from node 1 using the network API
network_3001 = scenario_3001.get_network()
shortest_path_tree = network_3001.shortest_path_tree(
    origin_node_id=1,
    link_costs='length',
    excluded_links=[network_3001.link(602, 601), network_3001.link(601, 602)],
    consider_turns=False
)

In [ ]:
# Fill in the @cost_from_1 values
for node, cost in shortest_path_tree.node_costs():
    node['@cost_from_1'] = cost

#publish the modified network
scenario_3001.publish_network(network_3001)

Go in the Desktop, open the general worksheet and visualize the new node extra attribute.

To see changes, make sure to use the Refresh Data button in Emme Desktop (Ctrl-R) or run the next cell.

In [ ]:
modeller.desktop.refresh_data() #preview of Desktop API

## 2.14 Speeding up network data processing with get_partial_network
The `scenario.get_partial_network(element_types, include_attributes)` method returns a network with a partial load of selected element types and attribute values. This method is similar to `get_network` but returns only partial network topology of a network on disk with limited access to elements and attributes. This is useful in suitable applications to minimize memory consumption and load time. 



The _element_types_ is a list of one or more network domain types from the Emme network hierarchy. Available values are `'MODE'`, `'TRANSIT_VEHICLE'`, `'NODE'`, `'LINK'`, `'TURN'`, `'TRANSIT_LINE'`, and `'TRANSIT_SEGMENT'`. Specifying a higher-level domain will automatically bring required dependencies in the network hierarchy (e.g. specifying `'LINK'` will also read in `'NODE'` and `'MODE'`).



If *include_attributes* is `True` the attributes values will also be loaded in the network for the specified network domain types in element_types. If *include_attributes* is `False` no attributes values will be loaded. The attributes will always be available on the network elements, but if the values are not loaded, the values for all elements will be set to the default value.



For our example, we will access the total number of boardings on each transit line. We do not need to read in the whole network to retrieve this information. Our strategy is to do a partial loading of the network to access transit line and transit segment elements (using `get_partial_network`) and to read-in only the required attribute values (using `get_attribute_values`, `set_attribute_values`).

Let us load the partial network with the required network elements:

In [ ]:
network = scenario2000.get_partial_network(
    element_types=['TRANSIT_SEGMENT', 'TRANSIT_LINE'],
    include_attributes=False
)

Print out the coordinates of the i node and j node for the first 10 links.

In [ ]:
link_iterator = network.links()
for i in range(10):
    link = link_iterator.next()
    print link, link.i_node.x, link.i_node.y, link.j_node.x, link.j_node.y

Links and nodes were imported as they are required to access transit line and transit segment elements, but their attributes values were not imported (the i node and j node coordinates are set to the default values of 0). Similarly, the transit segment number of boardings were not loaded:

In [ ]:
transit_segment_iterator = network.transit_segments()
for i in range(10):
    transit_segment = transit_segment_iterator.next()
    print transit_segment, transit_segment['transit_boardings']

We need to load in the transit segment result attribute values `transit_boardings`, by applying what we learnt in section <a href="#2.11-Fast-copy-of-attribute-values-from-scenario/network-to-scenario/network">2.11 Fast copy of attribute values from scenario/network to scenario/network</a>.

In [ ]:
seg_att_values = scenario2000.get_attribute_values('TRANSIT_SEGMENT', ['transit_boardings'])
network.set_attribute_values('TRANSIT_SEGMENT', ['transit_boardings'], seg_att_values)

We can now iterate over the transit lines and print out the total number of boardings for each transit line:

In [ ]:
report = []
for transit_line in network.transit_lines():
    total_boardings = 0
    for segment in transit_line.segments():
        total_boardings += segment.transit_boardings
    report.append('%s, %s\n' % (transit_line.id, total_boardings))
print ''.join(report)

Let us compare run times between the two following approaches:

- A: loading of the full network
- B: loading of a partial network

We will use the magic word `%%timeit`. It measures the execution time of the cell in a consistent way.

In [ ]:
%%timeit
# Method A
network =  scenario2000.get_network()

report = []
for transit_line in network.transit_lines():
    total_boardings = 0
    for segment in transit_line.segments():
        total_boardings += segment.transit_boardings
    report.append('%s, %s\n' % (transit_line.id, total_boardings))

In [ ]:
%%timeit
# Method B
network = scenario2000.get_partial_network(
    element_types=['TRANSIT_SEGMENT', 'TRANSIT_LINE'],
    include_attributes=False
)
seg_att_values = scenario2000.get_attribute_values('TRANSIT_SEGMENT', ['transit_boardings'])
network.set_attribute_values('TRANSIT_SEGMENT', ['transit_boardings'], seg_att_values)

report = []
for transit_line in network.transit_lines():
    total_boardings = 0
    for segment in transit_line.segments():
        total_boardings += segment.transit_boardings
    report.append('%s, %s\n' % (transit_line.id, total_boardings))

For our simple example, the method B (partial loading of the network) proved to be much faster than method A. The speedups you can achieve vary by application and network.